In [ ]:
%matplotlib inline

In [ ]:
from IPython.display import display, HTML
from ipywidgets import interact, Layout
import ipywidgets as widgets

In [ ]:
import os
import threading
import collections

In [ ]:
os.getpid(), threading.get_ident()

In [ ]:

import time
import datetime
import decimal
import warnings

import collections

from fast_trader.strategy import Strategy, get_strategy_instance
from fast_trader.dtp_quote import QuoteFeed
from fast_trader.utils import timeit

from math import ceil

In [ ]:
from data_provider.datafeed.universe import Universe

In [ ]:
out = widgets.Output()

In [ ]:


class TWAP(Strategy):
    
    def on_start(self):
        self.market_trades = collections.defaultdict(list)

    def on_market_trade(self, data):
        if data.nPrice > 0:
            self.market_trades[data.szCode].append(data)

    def on_trade_tmp(self, trade):
        pass
        print(trade.message)

    
    def on_order_tmp(self, order):
        self.logger.info(order)
        pass
        print('test:', order.message)
        time.sleep(1)
        with out:
            print(order.message)


    def on_order_cancelation(self, msg):
        print(msg.body)



In [ ]:
def test_time_weighted_ordering(total_quantity=1000000,
                                total_seconds=100,
                                interval=10):
    
    times = int(total_seconds / interval) + 1
    size_unit = ceil(total_quantity / 100 / times)
    # residule = total_quantity % times
    
    quantity_left = total_quantity
    for i in range(times):
        code = '600030'
        # price = ea.market_trades[code][-1].nPrice / 10000
        price = 9.9
        quantity = min(quantity_left, size_unit * 100)
        print('size:', quantity)
        ea.buy(code, price, quantity)
        quantity_left -= quantity
        
        if quantity_left <= 0:
            print('Finished', quantity_left)
            break
        time.sleep(interval)


In [ ]:
ea = get_strategy_instance(TWAP, 3)

from data_provider.datafeed.universe import Universe
sz50 = Universe().get_index_compose_weight_by_date('000016.SH', '20181109')
codes = sz50.securityId.tolist()
codes = [c[:6] for c in codes]

datasource = QuoteFeed('trade_feed')
datasource.subscribe(codes)

ea.add_datasource(datasource)

ea.start()

In [ ]:
ea.get_capital()

In [ ]:
def get_stock_name(code):
    if not hasattr(get_stock_name, '_pairs'):
        ipo_info = Universe().get_all_ipo_info()
        pairs = ipo_info[['name', 'ticker']].set_index(
            'ticker')['name'].to_dict()
        get_stock_name._pairs = pairs

    if '.' not in code:
        if code.startswith('6'):
            code += '.SH'
        else:
            code += '.SZ'
    try:
        return get_stock_name._pairs[code]
    except KeyError:
        return '未知代码'


In [ ]:
# mock_order_list = [{'order_exchange_id': '15961', 'order_original_id': '304067777', 'exchange': 1, 'code': '600030', 'name': '中信证券', 'price': '17.220', 'quantity': 91000, 'order_side': 1, 'order_type': 1, 'status': 4, 'order_time': '13:20:33', 'account_no': '021000062436', 'average_fill_price': '17.220', 'total_fill_quantity': 91000, 'total_fill_amount': '1567020.000', 'freeze_amount': '1571752.400', 'clear_amount': '-1571752.400', 'total_cancelled_quantity': 0, 'status_message': '全部成交'}, {'order_exchange_id': '15963', 'order_original_id': '304067778', 'exchange': 1, 'code': '600030', 'name': '中信证券', 'price': '17.220', 'quantity': 91000, 'order_side': 1, 'order_type': 1, 'status': 4, 'order_time': '13:20:43', 'account_no': '021000062436', 'average_fill_price': '17.220', 'total_fill_quantity': 91000, 'total_fill_amount': '1567020.000', 'freeze_amount': '1571752.400', 'clear_amount': '-1571752.400', 'total_cancelled_quantity': 0, 'status_message': '全部成交'}, {'order_exchange_id': '15971', 'order_original_id': '304067779', 'exchange': 1, 'code': '600030', 'name': '中信证券', 'price': '17.220', 'quantity': 91000, 'order_side': 1, 'order_type': 1, 'status': 4, 'order_time': '13:20:53', 'account_no': '021000062436', 'average_fill_price': '17.220', 'total_fill_quantity': 91000, 'total_fill_amount': '1567020.000', 'freeze_amount': '1571752.400', 'clear_amount': '-1571752.400', 'total_cancelled_quantity': 0, 'status_message': '全部成交'}, {'order_exchange_id': '15974', 'order_original_id': '304067780', 'exchange': 1, 'code': '600030', 'name': '中信证券', 'price': '17.220', 'quantity': 91000, 'order_side': 1, 'order_type': 1, 'status': 4, 'order_time': '13:21:03', 'account_no': '021000062436', 'average_fill_price': '17.220', 'total_fill_quantity': 91000, 'total_fill_amount': '1567020.000', 'freeze_amount': '1571752.400', 'clear_amount': '-1571752.400', 'total_cancelled_quantity': 0, 'status_message': '全部成交'}, {'order_exchange_id': '15980', 'order_original_id': '304067781', 'exchange': 1, 'code': '600030', 'name': '中信证券', 'price': '17.220', 'quantity': 91000, 'order_side': 1, 'order_type': 1, 'status': 4, 'order_time': '13:21:13', 'account_no': '021000062436', 'average_fill_price': '17.220', 'total_fill_quantity': 91000, 'total_fill_amount': '1567020.000', 'freeze_amount': '1571752.400', 'clear_amount': '-1571752.400', 'total_cancelled_quantity': 0, 'status_message': '全部成交'}, {'order_exchange_id': '15990', 'order_original_id': '304067782', 'exchange': 1, 'code': '600030', 'name': '中信证券', 'price': '17.220', 'quantity': 91000, 'order_side': 1, 'order_type': 1, 'status': 4, 'order_time': '13:21:23', 'account_no': '021000062436', 'average_fill_price': '17.220', 'total_fill_quantity': 91000, 'total_fill_amount': '1567020.000', 'freeze_amount': '1571752.400', 'clear_amount': '-1571752.400', 'total_cancelled_quantity': 0, 'status_message': '全部成交'}, {'order_exchange_id': '15998', 'order_original_id': '304067783', 'exchange': 1, 'code': '600030', 'name': '中信证券', 'price': '17.220', 'quantity': 91000, 'order_side': 1, 'order_type': 1, 'status': 4, 'order_time': '13:21:33', 'account_no': '021000062436', 'average_fill_price': '17.220', 'total_fill_quantity': 91000, 'total_fill_amount': '1567020.000', 'freeze_amount': '1571752.400', 'clear_amount': '-1571752.400', 'total_cancelled_quantity': 0, 'status_message': '全部成交'}, {'order_exchange_id': '16003', 'order_original_id': '304067784', 'exchange': 1, 'code': '600030', 'name': '中信证券', 'price': '17.220', 'quantity': 91000, 'order_side': 1, 'order_type': 1, 'status': 4, 'order_time': '13:21:44', 'account_no': '021000062436', 'average_fill_price': '17.220', 'total_fill_quantity': 91000, 'total_fill_amount': '1567020.000', 'freeze_amount': '1571752.400', 'clear_amount': '-1571752.400', 'total_cancelled_quantity': 0, 'status_message': '全部成交'}, {'order_exchange_id': '16007', 'order_original_id': '304067785', 'exchange': 1, 'code': '600030', 'name': '中信证券', 'price': '17.220', 'quantity': 91000, 'order_side': 1, 'order_type': 1, 'status': 4, 'order_time': '13:21:54', 'account_no': '021000062436', 'average_fill_price': '17.220', 'total_fill_quantity': 91000, 'total_fill_amount': '1567020.000', 'freeze_amount': '1571752.400', 'clear_amount': '-1571752.400', 'total_cancelled_quantity': 0, 'status_message': '全部成交'}, {'order_exchange_id': '16011', 'order_original_id': '304067786', 'exchange': 1, 'code': '600030', 'name': '中信证券', 'price': '17.220', 'quantity': 91000, 'order_side': 1, 'order_type': 1, 'status': 4, 'order_time': '13:22:04', 'account_no': '021000062436', 'average_fill_price': '17.220', 'total_fill_quantity': 91000, 'total_fill_amount': '1567020.000', 'freeze_amount': '1571752.400', 'clear_amount': '-1571752.400', 'total_cancelled_quantity': 0, 'status_message': '全部成交'}, {'order_exchange_id': '16018', 'order_original_id': '304067787', 'exchange': 1, 'code': '600030', 'name': '中信证券', 'price': '17.220', 'quantity': 90000, 'order_side': 1, 'order_type': 1, 'status': 4, 'order_time': '13:22:14', 'account_no': '021000062436', 'average_fill_price': '17.220', 'total_fill_quantity': 90000, 'total_fill_amount': '1549800.000', 'freeze_amount': '1554480.400', 'clear_amount': '-1554480.400', 'total_cancelled_quantity': 0, 'status_message': '全部成交'}]

# mock_position_list = [{'exchange': 1, 'code': '600027', 'name': '华电国际', 'balance': 2, 'available_quantity': 2, 'freeze_quantity': 0, 'buy_quantity': 0, 'sell_quantity': 0, 'market_value': '1639.800', 'cost': '-25.845'}, {'exchange': 1, 'code': '600187', 'name': '国中水务', 'balance': 2, 'available_quantity': 2, 'freeze_quantity': 0, 'buy_quantity': 0, 'sell_quantity': 0, 'market_value': '24.060', 'cost': '-20.225'}, {'exchange': 1, 'code': '600200', 'name': '江苏吴中', 'balance': 2, 'available_quantity': 2, 'freeze_quantity': 0, 'buy_quantity': 0, 'sell_quantity': 0, 'market_value': '32.320', 'cost': '-3.825'}, {'exchange': 1, 'code': '600209', 'name': '罗顿发展', 'balance': 1, 'available_quantity': 1, 'freeze_quantity': 0, 'buy_quantity': 0, 'sell_quantity': 0, 'market_value': '11.520', 'cost': '-505.370'}, {'exchange': 1, 'code': '600228', 'name': '昌九生化', 'balance': 2, 'available_quantity': 2, 'freeze_quantity': 0, 'buy_quantity': 0, 'sell_quantity': 0, 'market_value': '75.520', 'cost': '-5.485'}, {'exchange': 1, 'code': '600256', 'name': '广汇能源', 'balance': 22, 'available_quantity': 22, 'freeze_quantity': 0, 'buy_quantity': 0, 'sell_quantity': 0, 'market_value': '1162.480', 'cost': '7.186'}, {'exchange': 1, 'code': '600362', 'name': '江西铜业', 'balance': 2, 'available_quantity': 2, 'freeze_quantity': 0, 'buy_quantity': 0, 'sell_quantity': 0, 'market_value': '72.220', 'cost': '455.510'}, {'exchange': 1, 'code': '600470', 'name': '六国化工', 'balance': 2, 'available_quantity': 2, 'freeze_quantity': 0, 'buy_quantity': 0, 'sell_quantity': 0, 'market_value': '16.760', 'cost': '150.725'}, {'exchange': 1, 'code': '600487', 'name': '亨通光电', 'balance': 9, 'available_quantity': 9, 'freeze_quantity': 0, 'buy_quantity': 0, 'sell_quantity': 0, 'market_value': '227.880', 'cost': '-1.748'}, {'exchange': 1, 'code': '600536', 'name': '中国软件', 'balance': 101, 'available_quantity': 101, 'freeze_quantity': 0, 'buy_quantity': 0, 'sell_quantity': 0, 'market_value': '1460.460', 'cost': '13.800'}, {'exchange': 1, 'code': '600549', 'name': '厦门钨业', 'balance': 3, 'available_quantity': 3, 'freeze_quantity': 0, 'buy_quantity': 0, 'sell_quantity': 0, 'market_value': '116.310', 'cost': '116.460'}, {'exchange': 1, 'code': '600550', 'name': '天威保变', 'balance': 200, 'available_quantity': 200, 'freeze_quantity': 0, 'buy_quantity': 0, 'sell_quantity': 0, 'market_value': '1476.000', 'cost': '9.975'}, {'exchange': 1, 'code': '600552', 'name': '方兴科技', 'balance': 2, 'available_quantity': 2, 'freeze_quantity': 0, 'buy_quantity': 0, 'sell_quantity': 0, 'market_value': '39.940', 'cost': '124.345'}, {'exchange': 1, 'code': '600606', 'name': '金丰投资', 'balance': 100, 'available_quantity': 100, 'freeze_quantity': 0, 'buy_quantity': 0, 'sell_quantity': 0, 'market_value': '800.000', 'cost': '19.893'}, {'exchange': 1, 'code': '600633', 'name': '浙报传媒', 'balance': 4, 'available_quantity': 4, 'freeze_quantity': 0, 'buy_quantity': 0, 'sell_quantity': 0, 'market_value': '77.560', 'cost': '150.778'}, {'exchange': 1, 'code': '600640', 'name': '号百控股', 'balance': 103, 'available_quantity': 103, 'freeze_quantity': 0, 'buy_quantity': 0, 'sell_quantity': 0, 'market_value': '967.170', 'cost': '22.662'}, {'exchange': 1, 'code': '600643', 'name': '爱建股份', 'balance': 3, 'available_quantity': 3, 'freeze_quantity': 0, 'buy_quantity': 0, 'sell_quantity': 0, 'market_value': '19.410', 'cost': '-17.447'}, {'exchange': 1, 'code': '600678', 'name': '四川金顶', 'balance': 2, 'available_quantity': 2, 'freeze_quantity': 0, 'buy_quantity': 0, 'sell_quantity': 0, 'market_value': '13.240', 'cost': '-356.475'}, {'exchange': 1, 'code': '600756', 'name': '浪潮软件', 'balance': 2, 'available_quantity': 2, 'freeze_quantity': 0, 'buy_quantity': 0, 'sell_quantity': 0, 'market_value': '20.700', 'cost': '-6.870'}, {'exchange': 1, 'code': '600880', 'name': '博瑞传播', 'balance': 6, 'available_quantity': 6, 'freeze_quantity': 0, 'buy_quantity': 0, 'sell_quantity': 0, 'market_value': '68.640', 'cost': '-72.210'}, {'exchange': 1, 'code': '600976', 'name': '武汉健民', 'balance': 2, 'available_quantity': 2, 'freeze_quantity': 0, 'buy_quantity': 0, 'sell_quantity': 0, 'market_value': '40.060', 'cost': '-33.260'}, {'exchange': 1, 'code': '601390', 'name': '中国中铁', 'balance': 100, 'available_quantity': 100, 'freeze_quantity': 0, 'buy_quantity': 0, 'sell_quantity': 0, 'market_value': '193.000', 'cost': '11.920'}, {'exchange': 1, 'code': '601789', 'name': '宁波建工', 'balance': 2, 'available_quantity': 2, 'freeze_quantity': 0, 'buy_quantity': 0, 'sell_quantity': 0, 'market_value': '16.000', 'cost': '-1.800'}, {'exchange': 1, 'code': '601808', 'name': 'XD中海油', 'balance': 3, 'available_quantity': 3, 'freeze_quantity': 0, 'buy_quantity': 0, 'sell_quantity': 0, 'market_value': '42.810', 'cost': '-200.600'}, {'exchange': 1, 'code': '601992', 'name': '金隅股份', 'balance': 2, 'available_quantity': 2, 'freeze_quantity': 0, 'buy_quantity': 0, 'sell_quantity': 0, 'market_value': '15.540', 'cost': '15.950'}, {'exchange': 1, 'code': '603181', 'name': '皇马科技', 'balance': 200, 'available_quantity': 200, 'freeze_quantity': 0, 'buy_quantity': 0, 'sell_quantity': 0, 'market_value': '3088.000', 'cost': '21.055'}, {'exchange': 2, 'code': '000166', 'name': '申万宏源', 'balance': 135, 'available_quantity': 135, 'freeze_quantity': 0, 'buy_quantity': 0, 'sell_quantity': 0, 'market_value': '610.200', 'cost': '14.277'}, {'exchange': 2, 'code': '000401', 'name': '冀东水泥', 'balance': 100, 'available_quantity': 100, 'freeze_quantity': 0, 'buy_quantity': 0, 'sell_quantity': 0, 'market_value': '1067.000', 'cost': '17.599'}, {'exchange': 2, 'code': '000856', 'name': '冀东装备', 'balance': 100, 'available_quantity': 100, 'freeze_quantity': 0, 'buy_quantity': 0, 'sell_quantity': 0, 'market_value': '1124.000', 'cost': '43.913'}, {'exchange': 2, 'code': '000962', 'name': '东方钽业', 'balance': 100, 'available_quantity': 100, 'freeze_quantity': 0, 'buy_quantity': 0, 'sell_quantity': 0, 'market_value': '692.000', 'cost': '12.706'}, {'exchange': 2, 'code': '001965', 'name': '招商公路', 'balance': 1, 'available_quantity': 1, 'freeze_quantity': 0, 'buy_quantity': 0, 'sell_quantity': 0, 'market_value': '8.620', 'cost': '5.790'}, {'exchange': 2, 'code': '002024', 'name': '苏宁易购', 'balance': 2, 'available_quantity': 2, 'freeze_quantity': 0, 'buy_quantity': 0, 'sell_quantity': 0, 'market_value': '23.180', 'cost': '1.080'}, {'exchange': 2, 'code': '002036', 'name': '联创电子', 'balance': 100, 'available_quantity': 100, 'freeze_quantity': 0, 'buy_quantity': 0, 'sell_quantity': 0, 'market_value': '978.000', 'cost': '10.394'}, {'exchange': 2, 'code': '002080', 'name': '中材科技', 'balance': 1, 'available_quantity': 1, 'freeze_quantity': 0, 'buy_quantity': 0, 'sell_quantity': 0, 'market_value': '8.100', 'cost': '-267.160'}, {'exchange': 2, 'code': '002081', 'name': '金 螳 螂', 'balance': 12, 'available_quantity': 12, 'freeze_quantity': 0, 'buy_quantity': 0, 'sell_quantity': 0, 'market_value': '107.760', 'cost': '-12.111'}, {'exchange': 2, 'code': '002085', 'name': '万丰奥威', 'balance': 5, 'available_quantity': 5, 'freeze_quantity': 0, 'buy_quantity': 0, 'sell_quantity': 0, 'market_value': '40.000', 'cost': '-2.514'}, {'exchange': 2, 'code': '002091', 'name': '江苏国泰', 'balance': 6, 'available_quantity': 6, 'freeze_quantity': 0, 'buy_quantity': 0, 'sell_quantity': 0, 'market_value': '33.600', 'cost': '-121.015'}, {'exchange': 2, 'code': '002140', 'name': '东华科技', 'balance': 2, 'available_quantity': 2, 'freeze_quantity': 0, 'buy_quantity': 0, 'sell_quantity': 0, 'market_value': '12.720', 'cost': '-71.470'}, {'exchange': 2, 'code': '002195', 'name': '二三四五', 'balance': 70, 'available_quantity': 70, 'freeze_quantity': 0, 'buy_quantity': 0, 'sell_quantity': 0, 'market_value': '300.300', 'cost': '51.829'}, {'exchange': 2, 'code': '002230', 'name': '科大讯飞', 'balance': 5, 'available_quantity': 5, 'freeze_quantity': 0, 'buy_quantity': 0, 'sell_quantity': 0, 'market_value': '121.450', 'cost': '-167.982'}, {'exchange': 2, 'code': '002235', 'name': '安妮股份', 'balance': 503, 'available_quantity': 503, 'freeze_quantity': 0, 'buy_quantity': 0, 'sell_quantity': 0, 'market_value': '3143.750', 'cost': '10.691'}, {'exchange': 2, 'code': '002259', 'name': 'ST升达', 'balance': 1, 'available_quantity': 1, 'freeze_quantity': 0, 'buy_quantity': 0, 'sell_quantity': 0, 'market_value': '2.860', 'cost': '-261.400'}, {'exchange': 2, 'code': '002263', 'name': '*ST东南', 'balance': 300, 'available_quantity': 300, 'freeze_quantity': 0, 'buy_quantity': 0, 'sell_quantity': 0, 'market_value': '585.000', 'cost': '4.503'}, {'exchange': 2, 'code': '002280', 'name': '联络互动', 'balance': 100, 'available_quantity': 100, 'freeze_quantity': 0, 'buy_quantity': 0, 'sell_quantity': 0, 'market_value': '447.000', 'cost': '31.551'}, {'exchange': 2, 'code': '002294', 'name': '信立泰', 'balance': 3, 'available_quantity': 3, 'freeze_quantity': 0, 'buy_quantity': 0, 'sell_quantity': 0, 'market_value': '77.400', 'cost': '7.247'}, {'exchange': 2, 'code': '002310', 'name': '东方园林', 'balance': 8, 'available_quantity': 8, 'freeze_quantity': 0, 'buy_quantity': 0, 'sell_quantity': 0, 'market_value': '80.960', 'cost': '-118.760'}, {'exchange': 2, 'code': '002346', 'name': '柘中股份', 'balance': 100, 'available_quantity': 100, 'freeze_quantity': 0, 'buy_quantity': 0, 'sell_quantity': 0, 'market_value': '1173.000', 'cost': '20.800'}, {'exchange': 2, 'code': '002375', 'name': '亚厦股份', 'balance': 8, 'available_quantity': 8, 'freeze_quantity': 0, 'buy_quantity': 0, 'sell_quantity': 0, 'market_value': '44.240', 'cost': '-58.284'}, {'exchange': 2, 'code': '002385', 'name': '大北农', 'balance': 3, 'available_quantity': 3, 'freeze_quantity': 0, 'buy_quantity': 0, 'sell_quantity': 0, 'market_value': '10.650', 'cost': '21.520'}, {'exchange': 2, 'code': '002397', 'name': '梦洁股份', 'balance': 9, 'available_quantity': 9, 'freeze_quantity': 0, 'buy_quantity': 0, 'sell_quantity': 0, 'market_value': '41.130', 'cost': '-28.819'}, {'exchange': 2, 'code': '002477', 'name': '雏鹰农牧', 'balance': 3, 'available_quantity': 3, 'freeze_quantity': 0, 'buy_quantity': 0, 'sell_quantity': 0, 'market_value': '5.730', 'cost': '-240.030'}, {'exchange': 2, 'code': '002635', 'name': '安洁科技', 'balance': 209, 'available_quantity': 209, 'freeze_quantity': 0, 'buy_quantity': 0, 'sell_quantity': 0, 'market_value': '2597.870', 'cost': '34.908'}, {'exchange': 2, 'code': '002638', 'name': '勤上股份', 'balance': 300, 'available_quantity': 300, 'freeze_quantity': 0, 'buy_quantity': 0, 'sell_quantity': 0, 'market_value': '933.000', 'cost': '5.787'}, {'exchange': 2, 'code': '002723', 'name': '金莱特', 'balance': 4, 'available_quantity': 4, 'freeze_quantity': 0, 'buy_quantity': 0, 'sell_quantity': 0, 'market_value': '41.680', 'cost': '-114.943'}, {'exchange': 2, 'code': '002746', 'name': '仙坛股份', 'balance': 100, 'available_quantity': 100, 'freeze_quantity': 0, 'buy_quantity': 0, 'sell_quantity': 0, 'market_value': '1501.000', 'cost': '22.110'}, {'exchange': 2, 'code': '002774', 'name': '快意电梯', 'balance': 100, 'available_quantity': 100, 'freeze_quantity': 0, 'buy_quantity': 0, 'sell_quantity': 0, 'market_value': '936.000', 'cost': '13.060'}, {'exchange': 2, 'code': '300020', 'name': '银江股份', 'balance': 100, 'available_quantity': 100, 'freeze_quantity': 0, 'buy_quantity': 0, 'sell_quantity': 0, 'market_value': '696.000', 'cost': '21.559'}, {'exchange': 2, 'code': '300101', 'name': '振芯科技', 'balance': 4, 'available_quantity': 4, 'freeze_quantity': 0, 'buy_quantity': 0, 'sell_quantity': 0, 'market_value': '43.440', 'cost': '-40.540'}, {'exchange': 2, 'code': '300104', 'name': '乐视网', 'balance': 800, 'available_quantity': 800, 'freeze_quantity': 0, 'buy_quantity': 0, 'sell_quantity': 0, 'market_value': '2912.000', 'cost': '16.249'}, {'exchange': 2, 'code': '300137', 'name': '先河环保', 'balance': 3, 'available_quantity': 3, 'freeze_quantity': 0, 'buy_quantity': 0, 'sell_quantity': 0, 'market_value': '24.930', 'cost': '-125.750'}, {'exchange': 2, 'code': '300157', 'name': '恒泰艾普', 'balance': 3, 'available_quantity': 3, 'freeze_quantity': 0, 'buy_quantity': 0, 'sell_quantity': 0, 'market_value': '14.550', 'cost': '-292.343'}, {'exchange': 2, 'code': '300183', 'name': '东软载波', 'balance': 2, 'available_quantity': 2, 'freeze_quantity': 0, 'buy_quantity': 0, 'sell_quantity': 0, 'market_value': '23.880', 'cost': '-35.795'}, {'exchange': 2, 'code': '300205', 'name': '天喻信息', 'balance': 2, 'available_quantity': 2, 'freeze_quantity': 0, 'buy_quantity': 0, 'sell_quantity': 0, 'market_value': '16.340', 'cost': '501.270'}, {'exchange': 2, 'code': '300209', 'name': '天泽信息', 'balance': 100, 'available_quantity': 100, 'freeze_quantity': 0, 'buy_quantity': 0, 'sell_quantity': 0, 'market_value': '1158.000', 'cost': '18.300'}, {'exchange': 2, 'code': '300229', 'name': '拓尔思', 'balance': 100, 'available_quantity': 100, 'freeze_quantity': 0, 'buy_quantity': 0, 'sell_quantity': 0, 'market_value': '947.000', 'cost': '56.907'}, {'exchange': 2, 'code': '300234', 'name': '开尔新材', 'balance': 2, 'available_quantity': 2, 'freeze_quantity': 0, 'buy_quantity': 0, 'sell_quantity': 0, 'market_value': '13.100', 'cost': '1961.865'}, {'exchange': 2, 'code': '300305', 'name': '裕兴股份', 'balance': 4, 'available_quantity': 4, 'freeze_quantity': 0, 'buy_quantity': 0, 'sell_quantity': 0, 'market_value': '26.920', 'cost': '-40.167'}, {'exchange': 2, 'code': '300311', 'name': '任子行', 'balance': 50, 'available_quantity': 50, 'freeze_quantity': 0, 'buy_quantity': 0, 'sell_quantity': 0, 'market_value': '327.000', 'cost': '90.652'}, {'exchange': 2, 'code': '300340', 'name': '科恒股份', 'balance': 2, 'available_quantity': 2, 'freeze_quantity': 0, 'buy_quantity': 0, 'sell_quantity': 0, 'market_value': '28.880', 'cost': '424.605'}, {'exchange': 2, 'code': '300352', 'name': '北信源', 'balance': 5, 'available_quantity': 5, 'freeze_quantity': 0, 'buy_quantity': 0, 'sell_quantity': 0, 'market_value': '18.200', 'cost': '175.170'}, {'exchange': 2, 'code': '300358', 'name': '楚天科技', 'balance': 100, 'available_quantity': 100, 'freeze_quantity': 0, 'buy_quantity': 0, 'sell_quantity': 0, 'market_value': '830.000', 'cost': '12.190'}, {'exchange': 2, 'code': '300433', 'name': '蓝思科技', 'balance': 117, 'available_quantity': 117, 'freeze_quantity': 0, 'buy_quantity': 0, 'sell_quantity': 0, 'market_value': '988.650', 'cost': '27.250'}, {'exchange': 2, 'code': '300708', 'name': '聚灿光电', 'balance': 100, 'available_quantity': 100, 'freeze_quantity': 0, 'buy_quantity': 0, 'sell_quantity': 0, 'market_value': '1239.000', 'cost': '28.210'}, {'exchange': 1, 'code': '600030', 'name': '中信证券', 'balance': 1000000, 'available_quantity': 0, 'freeze_quantity': 0, 'buy_quantity': 1000000, 'sell_quantity': 0, 'market_value': '3600000000.000', 'cost': '17.272'}]

# mock_trade_list = [{'fill_exchange_id': 'b093910000000002', 'fill_time': '09:39:10', 'fill_status': 1, 'fill_price': '10.700', 'fill_quantity': 3000, 'fill_amount': '32100.000', 'clear_amount': '-32196.300', 'order_exchange_id': '1', 'order_original_id': '943938365', 'exchange': 2, 'code': '000001', 'name': '平安银行', 'order_side': 1}, {'fill_exchange_id': 'b094523000000006', 'fill_time': '09:45:23', 'fill_status': 1, 'fill_price': '10.600', 'fill_quantity': 14000, 'fill_amount': '148400.000', 'clear_amount': '-148845.200', 'order_exchange_id': '6', 'order_original_id': '943938366', 'exchange': 2, 'code': '000001', 'name': '平安银行', 'order_side': 1}, {'fill_exchange_id': 'b095146000002168', 'fill_time': '09:51:46', 'fill_status': 1, 'fill_price': '2.000', 'fill_quantity': 21000, 'fill_amount': '42000.000', 'clear_amount': '-42126.000', 'order_exchange_id': '2171', 'order_original_id': '943938367', 'exchange': 2, 'code': '300104', 'name': '乐视网', 'order_side': 1}, {'fill_exchange_id': '17747', 'fill_time': '10:13:26', 'fill_status': 1, 'fill_price': '560.000', 'fill_quantity': 6000, 'fill_amount': '3360000.000', 'clear_amount': '-3370147.200', 'order_exchange_id': '33355', 'order_original_id': '943938368', 'exchange': 1, 'code': '600519', 'name': '贵州茅台', 'order_side': 1}, {'fill_exchange_id': '40031', 'fill_time': '11:08:25', 'fill_status': 1, 'fill_price': '565.000', 'fill_quantity': 6000, 'fill_amount': '3390000.000', 'clear_amount': '-3400237.800', 'order_exchange_id': '72325', 'order_original_id': '957576377', 'exchange': 1, 'code': '600519', 'name': '贵州茅台', 'order_side': 1}, {'fill_exchange_id': '57692000', 'fill_time': '11:16:31', 'fill_status': 1, 'fill_price': '559.000', 'fill_quantity': 11000, 'fill_amount': '6149000.000', 'clear_amount': '-6167569.980', 'order_exchange_id': '102565', 'order_original_id': '957576378', 'exchange': 1, 'code': '600519', 'name': '贵州茅台', 'order_side': 1}, {'fill_exchange_id': '63273000', 'fill_time': '13:20:32', 'fill_status': 1, 'fill_price': '557.000', 'fill_quantity': 15000, 'fill_amount': '8355000.000', 'clear_amount': '-8380232.100', 'order_exchange_id': '109406', 'order_original_id': '979381787', 'exchange': 1, 'code': '600519', 'name': '贵州茅台', 'order_side': 1}, {'fill_exchange_id': 'b135602000013719', 'fill_time': '13:56:02', 'fill_status': 1, 'fill_price': '3.000', 'fill_quantity': 20000, 'fill_amount': '60000.000', 'clear_amount': '-60180.000', 'order_exchange_id': '123011', 'order_original_id': '123935159', 'exchange': 2, 'code': '300104', 'name': '乐视网', 'order_side': 1}, {'fill_exchange_id': '72943000', 'fill_time': '13:51:05', 'fill_status': 1, 'fill_price': '5.300', 'fill_quantity': 20000, 'fill_amount': '106000.000', 'clear_amount': '-106320.120', 'order_exchange_id': '126173', 'order_original_id': '123935160', 'exchange': 1, 'code': '600668', 'name': '尖峰集团', 'order_side': 1}, {'fill_exchange_id': '76740000', 'fill_time': '13:53:38', 'fill_status': 1, 'fill_price': '5.300', 'fill_quantity': 100000, 'fill_amount': '530000.000', 'clear_amount': '-531600.600', 'order_exchange_id': '132141', 'order_original_id': '123935161', 'exchange': 1, 'code': '600668', 'name': '尖峰集团', 'order_side': 1}, {'fill_exchange_id': '77373000', 'fill_time': '13:55:41', 'fill_status': 1, 'fill_price': '5.300', 'fill_quantity': 100000, 'fill_amount': '530000.000', 'clear_amount': '-531600.600', 'order_exchange_id': '133215', 'order_original_id': '123935162', 'exchange': 1, 'code': '601668', 'name': '中国建筑', 'order_side': 1}]

# moci_capital = \
# {'account_no': '021000062436',
#  'balance': '1000006503.160',
#  'available': '980442001.960',
#  'freeze': '19564501.200',
#  'securities': '3982406736.490',
#  'total': '4963241899.770'}

# class EA(object):
#     def get_capital(self):
#         return mock_capital
#     def get_orders(self):
#         return mock_order_list[:10]
#     def get_open_orders(self):
#         return mock_order_list[:10]
#     def get_trades(self):
#         return mock_trade_list[:10]
#     def get_positions(self):
#         return mock_position_list[:10]
# ea = EA()

In [ ]:
def create_trading_panel(ea):
    stock_code = widgets.Text(description='股票代码', continuous_update=True)
    stock_name = widgets.Text(description='股票名称', continuous_update=False)
    order_price = widgets.BoundedFloatText(description='价格', min=0.01, max=100000,
                                           step=0.01, continuous_update=False)
    stock_quantity = widgets.BoundedIntText(description='数量', min=100, max=1000000000000,
                                            step=100, continuous_update=False)
    order_side = widgets.Dropdown(description='买卖方向', continuous_update=False)
    order_side.options = [('买入', 1), ('卖出', 2)]

    confirm_btn = widgets.Button(description='下单')



    panel = box = widgets.Box([stock_code, stock_name, order_price,
                       stock_quantity, order_side, confirm_btn, out])

    def fill_stock_name(change):
        try:
            value = change['new']
            if len(value) >= 6:
                name = get_stock_name(value)
                stock_name.value = name
        except Exception as e:
            with out:
                print(e)

    def insert_order(b):
        with out:
            try:
                code = box.children[0].value
                price = box.children[2].value
                quantity = box.children[3].value
                order_side = box.children[4].value

                err_msg = None
                if price:
                    if decimal.Decimal(str(price)).as_tuple().exponent < -2:
                        err_msg = '最多保留小数点后两位！'

                if err_msg is not None:
                    raise Exception(err_msg)

                if order_side == 1:
                    ret = ea.buy(code, price, quantity)
                else:
                    ret = ea.sell(code, price, quantity)

            except Exception as e:
                with out:
                    display(HTML('<p style=color:red>{}<p>'.format(e)))

            else:
                pass

    confirm_btn.on_click(insert_order)

    stock_code.observe(fill_stock_name, names='value')

    confirm_btn.layout = Layout(
        width='212px',
        margin='10px 0px 10px 90px')

    panel.layout = Layout(
        display='flex',
        flex_flow='column',
        align_items='stretch',
        border=None,
        width='40%')

    return panel

In [ ]:
trading_panel = create_trading_panel(ea)
trading_panel

In [ ]:
def index_table(panel, row, column, as_widget=False):
    item = panel.children[row].children[column]
    if as_widget:
        return item
    return item.description

In [ ]:
ea.positions['002230']

In [ ]:
account_label_dict = collections.OrderedDict([
    ('账户', 'account_no'),
    ('余额', 'balance'),
    ('可用资金', 'available'),
    ('冻结金额', 'freeze'),
    ('证券市值', 'securities'),
    ('总资产', 'total')
])

trade_label_dict = collections.OrderedDict([
    ('证券代码', 'code'),
    ('证券名称', 'name'),
    ('交易方向', 'order_side'),
    ('成交价格', 'fill_price'),
    ('成交数量', 'fill_quantity'),
    ('状态', 'fill_status'),
    ('成交时间', 'fill_time'),
    ('成交金额', 'fill_amount'),
    ('清算资金', 'clear_amount'),
    ('客户委托编号', 'order_original_id'),
    ('交易所委托编号', 'order_exchange_id'),
    ('交易所成交编号', 'fill_exchange_id'),
    ('交易所', 'exchange')])
        
order_label_dict = collections.OrderedDict([
    ('证券代码', 'code'),
    ('证券名称', 'name'),
    ('委托价格', 'price'),
    ('委托数量', 'quantity'),
    ('已成交数量', 'total_fill_quantity'),
    ('交易方向', 'order_side'),
    ('报价类型', 'order_type'),
    ('状态', 'status_message'),
    ('委托时间', 'order_time'),
    ('冻结资金', 'freeze_amount'),
    ('交易所委托编号', 'order_exchange_id'),
    ('客户委托编号', 'order_original_id'),
    ('交易所', 'exchange')])

# position_label_dict = collections.OrderedDict([
#     ('证券代码', 'code'),
#     ('证券名称', 'name'),
#     ('交易所', 'exchange'),
#     ('持仓', 'balance'),
#     ('可用数量', 'available_quantity'),
#     ('冻结数量', 'freeze_quantity'),
#     ('买入数量', 'buy_quantity'),
#     ('卖出数量', 'sell_quantity'),
#     ('市值', 'market_value'),
#     ('成本价', 'cost')])

position_label_dict = collections.OrderedDict([
    ('证券代码', 'code'),
    ('证券名称', 'name'),
    ('持仓', 'quantity'),
    ('成本价', 'cost_price'),
    ('可用数量', 'available_quantity'),
    ('冻结数量', 'freeze_quantity'),
    ('买入数量', 'buy_quantity'),
    ('卖出数量', 'sell_quantity'),
    ('市值', 'market_value'),
    ('交易所', 'exchange')])

unit_cls = widgets.Button
# unit_cls = widgets.Label

# ATTR = 'value'
ATTR = 'description'

def apply_style(**kwargs):
    def decorator(func):
        def wrapper(*args, **kw):
            obj = func(*args, **kw)
            for k, v in kwargs.items():
                setattr(obj.layout, k, v)
            return obj
        return wrapper
    return decorator

class Dashboard(object):
    


    def __init__(self, ea):
        
        self._running = False
        self.ea = ea
        
        self.trading_panel = None
        self.account_panel = None
        self.trade_panel = None
        self.order_panel = None
        self.position_panel = None

    def is_alive(self):
        return self._running

    @staticmethod
    def has_item(panel, ident):
        for child in panel.children:
            if child.ident == ident:
                return True
        return False

    @staticmethod
    def create_panel(label_dict):
        headers = [unit_cls(**{ATTR: label, 'button_style': ''})
                   for label in label_dict]
        [setattr(el.style, 'font_weight', 'bold') for el in headers]
        header_box = widgets.HBox(headers)
        header_box.ident = 'header'
#         placeholder = widgets.HTML('<hr>')
#         placeholder.ident = 'placeholder'
#         panel = widgets.VBox([header_box, placeholder])
        panel = widgets.VBox([header_box])
        return panel

    @staticmethod
    def add_panel_entry(panel, label_dict, entry, ident):
        def to_widget(entry):
            item = widgets.HBox([unit_cls(**{ATTR: str(entry.get(v, '-')),
                                             'style': {'button_color': 'white'}})
                                 for v in label_dict.values()])
            return item
        item = to_widget(entry)
        item.ident = ident
        panel.children = (*panel.children[:1], item, *panel.children[1:])

    @staticmethod
    def update_panel_entry(panel, label_dict, entry, ident, upsert=True):
        
        if upsert:
            if not Dashboard.has_item(panel, ident):
                Dashboard.add_panel_entry(panel, label_dict, entry, ident)
                return
                
        fields = list(label_dict.values())
        for child in panel.children:
            if child.ident == ident:
                for i, el in enumerate(child.children):
                    key = fields[i]
                    if key in entry:
                        setattr(el, ATTR, str(entry[key]))
                break
    
    @apply_style(width='fit-content')
    def create_account_panel(self):
        self.account_panel = self.create_panel(account_label_dict)
        return self.account_panel
    
    def update_account_panel(self, entry):
        ident = entry['account_no']
        self.update_panel_entry(
            self.account_panel,
            account_label_dict,
            entry=entry,
            ident=ident)

    def refresh_capital(self):
        capital = self.ea.get_capital()
        self.update_account_panel(capital)
    
    @apply_style(width='fit-content')
    def create_trade_panel(self):
        self.trade_panel = self.create_panel(trade_label_dict)
        return self.trade_panel

    @timeit
    def update_trade_panel(self, entry):
        ident = entry['order_original_id']
        self.update_panel_entry(
            self.trade_panel,
            trade_label_dict,
            entry=entry,
            ident=ident)
        
    def refresh_trade_panel(self):
        trades = self.ea.get_trades()
        for trade in trades:
            ident = trade['order_original_id']
            if self.has_item(self.trade_panel, ident):
                continue
            self.update_trade_panel(trade)

    @apply_style(width='fit-content')
    def create_position_panel(self):
        self.position_panel = self.create_panel(position_label_dict)
        return self.position_panel

    @timeit
    def update_position_panel(self, entry):
        ident = entry['code']
        self.update_panel_entry(
            self.position_panel,
            position_label_dict,
            entry=entry,
            ident=ident)
        
    def refresh_position_panel(self):
        positions = self.ea.get_positions()
        for pos in positions:
            ident = pos['code']
#             if self.has_item(self.position_panel, ident):
#                 continue
            self.update_position_panel(pos)
    
    @apply_style(width='fit-content')
    def create_order_panel(self):
        self.order_panel = self.create_panel(order_label_dict)
        return self.order_panel

    @timeit
    def update_order_panel(self, entry):
        ident = entry['order_original_id']
        self.update_panel_entry(
            self.order_panel,
            order_label_dict,
            entry=entry,
            ident=ident)
        
    def refresh_order_panel(self):
        orders = self.ea.get_open_orders()
        valid_idents = ['header']
        for order in orders:
            ident = order['order_original_id']
            valid_idents.append(ident)
            if self.has_item(self.order_panel, ident):
                continue
            self.update_order_panel(order)
        
#         children_to_remove = []
#         for child in self.order_panel.children[1:]:
#             if child.ident not in valid_idents:
#                 children_to_remove.append(child)
        self.order_panel.children = [child for child in self.order_panel.children 
                                     if child.ident in valid_idents]

    def run(self):
        self._running = True
        while self._running:

            time.sleep(0.5)
            if self.account_panel:
                self.refresh_capital()
            if self.trade_panel:
                self.refresh_trade_panel()
            if self.position_panel:
                self.refresh_position_panel()
            if self.order_panel:
                self.refresh_order_panel()
    
    def refresh_dashboard(self):
        self._thread = threading.Thread(target=self.run)
        self._thread.start()
            

In [ ]:
da = Dashboard(ea)

### 账户

In [ ]:
account_panel = da.create_account_panel()
account_panel

In [ ]:
da.refresh_capital()

### 委托列表

In [ ]:
order_panel = da.create_order_panel()
order_panel

In [ ]:
# da.refresh_order_panel()

### 持仓列表

In [ ]:
position_panel = da.create_position_panel()
position_panel

In [ ]:
da.refresh_position_panel()

### 成交列表

In [ ]:
trade_panel = da.create_trade_panel()
trade_panel

In [ ]:
# da.refresh_trade_panel()

In [ ]:
da.refresh_dashboard()

In [ ]:
ea.positions['600016']